In [1]:
"""Package Loading"""
"""General"""
from webscraping_dictionaries import *
from webscraping_functions import *
import time
from dateutil import relativedelta
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By  
import requests
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import ssl
import os
#import dotenv
"""Chrome Options"""
#from selenium.webdriver import Chrome 
#from selenium.webdriver.chrome.service import Service 
#from webdriver_manager.chrome import ChromeDriverManager

"""Firefox Options"""
#from selenium.webdriver import Firefox
#from selenium.webdriver.firefox.service import Service
#from webdriver_manager.firefox import GeckoDriverManager


"""Initialize the driver"""

"""Driver Setup"""

#following steps from here: https://www.zenrows.com/blog/scraping-javascript-rendered-web-pages#installing-the-requirements
#critical help using the click method from here https://www.geeksforgeeks.org/click-element-method-selenium-python/

"""Chrome Version"""
#options = webdriver.ChromeOptions()
#options.headless = True
#options.page_load_strategy = 'none' 
#chrome_path = ChromeDriverManager().install()
#chrome_service = Service(chrome_path)
#driver = Chrome(options=options, service=chrome_service) 
#driver.implicitly_wait(5)

"""Firefox Version"""  
#options = webdriver.FirefoxOptions()
#options.headless = True
#options.add_argument("--headless=new")
#options.page_load_strategy = 'none' 
#firefox_path = GeckoDriverManager().install() 
#firefox_service = Service(firefox_path)
#driver = Firefox(options=options, service=firefox_service)
#driver.implicitly_wait(5)

'Firefox Version'

"""DEBUGGING SANDBOX"""

In [ ]:
#table this for now, since we still want to check for comp plan and ordinance keywords anyway
"Charlotte County Solar Development Page"
def charlotte_county_solar_development():
    driver.get("https://www.charlottecountyva.gov/departments/planning___zoning/solar_development.php")
    time.sleep(1)
    table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")
    

In [ ]:
#come back to this another day
all_dictionaries=[agendacenter_dictionary,boarddocs_dictionary,civicclerk_dictionary,civicweb_dictionary,document_center_dictionary,escribe_dictionary,event_list_dictionary,granicus_dictionary,granicus_2_dictionary,laserfiche_dictionary,legistar_dictionary,meetingstable_dictionary,novusagenda_dictionary,php_table_dictionary,primegov_dictionary,county_dictionary_single_variable,county_dictionary_double_variable,city_dictionary_single_variable,city_dictionary_double_variable]

for dictionary in all_dictionaries:
    for entry in dictionary:
        print(entry)

In [ ]:
"""Essex County PC"""
def essex_pc(url):
    driver.get(url)
    time.sleep(5)
    messages = []
    all_rows = driver.find_elements(By.CSS_SELECTOR,"div[class*='views-row'")
    future_meetings = []
    for item in all_rows:
        try:
            future_meeting = check_meeting_date(item.text.split("-")[0])
            if future_meeting==True:
                future_meetings.append(item)
        except:
            continue
    agenda_links = []
    for item in future_meetings:
        try:
            agenda_links.append(item.find_element(By.CSS_SELECTOR,"a[title*=Agenda").get_attribute("href"))
        except:
            #throws an error of no links are posted yet, so do this and keep going to keep the indices aligned
            agenda_links.append(None)
    for item in agenda_links:
        if item != None:
            driver.get(item)
            time.sleep(2)
            agenda_content = driver.find_elements(By.CSS_SELECTOR,"div[class*=textLayer")
            agenda_search = search_agenda_for_keywords(agenda_content)
            if agenda_search != []:
                messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for Essex County Planning Commission. " + item)
    return messages

In [55]:
driver.get("https://www.essex-virginia.org/meetings?date_filter%5Bvalue%5D%5Bmonth%5D=1&date_filter%5Bvalue%5D%5Bday%5D=1&date_filter%5Bvalue%5D%5Byear%5D=2023&date_filter_1%5Bvalue%5D%5Bmonth%5D=12&date_filter_1%5Bvalue%5D%5Bday%5D=31&date_filter_1%5Bvalue%5D%5Byear%5D=2023&field_microsite_tid=All&field_microsite_tid_1=28")

In [61]:
all_rows = driver.find_elements(By.CSS_SELECTOR,"div[class*='views-row'")

In [69]:
    future_meetings = []
    for item in all_rows:
        try:
            future_meeting = check_meeting_date(item.text.split("-")[0])
            if future_meeting==True:
                future_meetings.append(item)
        except:
            continue

In [70]:
future_meetings

[<selenium.webdriver.remote.webelement.WebElement (session="33f1d0f5-170d-444a-98d5-6a75f9e26b60", element="5e0f3245-226f-4405-a4bd-d59783f86efc")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33f1d0f5-170d-444a-98d5-6a75f9e26b60", element="cb5a3642-492a-4cc3-ae9f-ceec00b691a4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33f1d0f5-170d-444a-98d5-6a75f9e26b60", element="515f65c1-16fe-4e55-9dcd-4bfaca671f23")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33f1d0f5-170d-444a-98d5-6a75f9e26b60", element="a3676d46-d2b6-4b56-8ec0-a3681bdb2ed0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33f1d0f5-170d-444a-98d5-6a75f9e26b60", element="cf5a3770-65e8-431d-b8a9-b5a02026424d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33f1d0f5-170d-444a-98d5-6a75f9e26b60", element="06cef275-75e3-4a07-8e5d-d4aacaa5dfd4")>,
 <selenium.webdriver.remote.webelement.WebElement (session="33f1d0f5-170d-444a-98d5-6a75f9e26b60", element